In [2]:
import pyAgrum.lib.notebook as gnb
import pyAgrum as gum
import cProfile
import random
import sys
import pyAgrum.lib.ipython as gnbipy
import numpy as np
import itertools
sys.path.append("C:\\Users\\zlhyl\\OneDrive\\Belgeler\\PhD")


    
from SDM_Functions_v02 import (
    update_decision_node_options,add_equal_probabilities_for_new_option,
    show_decision_utilities3,compare_final_decisions,
    add_nodes_from_model2_to_model1,arc_transfer,
    fill_model1_cpts_with_noisy_data,add_noise_to_cpt,calculate_utility_values,print_all_utilitiesX,transfer_parent_node_parameter,
    add_noise_to_increments,extract_parameters,delete_unmatched_arcs,find_matched_parents,get_utility_parents,find_unmatched_parents,
    has_empty_cpts,print_all_cpts,add_cpt_from_pool,selected_cpt_difficulty, selected_cpt_friends
)

from advisor_model_v05 import create_advisor_model
from student_model_v06 import create_student_model


advisor_model = create_advisor_model()
student_model = create_student_model()

parametersA = np.array([
    ['prerequisite', 0.6, 50],
    ['workload', 0.3, 30],
    ['career_development', 0.1, 10]
])

parametersS = np.array([
    ['Grade', 0.2, 60],
    ['friends', 0.3, 40],
    ['workload', 0.5, 10]
])


calculate_utility_values(advisor_model, 'Advisor Utility', parametersA)
calculate_utility_values(student_model, 'Student Utility', parametersS)

#gnb.sideBySide(advisor_model,student_model)

#utilities_model_student2, max_expected_utility_student2,_ = show_decision_utilities3(student_model)                
#utilities_model_advisor2, max_expected_utility_advisor2,_ = show_decision_utilities3(advisor_model)   
#result_df2 = compare_final_decisions(utilities_model_student2, utilities_model_advisor2, threshold=5)
#print(result_df2)



def decision_alternative_transfer(model1, model2, decision_node):
    model1,decision_node = update_decision_node_options(model1, model2, decision_node) 
       
    function_used = ""
    if random.random() < 0.5:
        add_equal_probabilities_for_new_option(model1)
        function_used = "add_equal_probabilities_for_new_option"
    else:
        add_cpt_from_pool(model1, selected_cpt_difficulty, selected_cpt_friends)
        function_used = "add_cpt_from_pool"
    
    return model1
    

def chance_node_transfer(model1, model2, chance_node):
    model1, chance_node = add_nodes_from_model2_to_model1(model1, model2, chance_node)
    model1 = arc_transfer(model1, model2, chance_node)
    model1 = fill_model1_cpts_with_noisy_data(model1, model2, chance_node)

    return model1

    
def preference_transfer(model1, model2, parameters1, parameters2, parent_node_name):
    utility_node_name_model1 = next(node for node in model1.names() if model1.isUtilityNode(node))
    utility_node_name_model2 = next(node for node in model2.names() if model2.isUtilityNode(node))
    
    model1 = chance_node_transfer(model1, model2, parent_node_name)
    updated_parameters1 = transfer_parent_node_parameter(parameters1, parameters2, parent_node_name)
    noisy_parameters1 = add_noise_to_increments(updated_parameters1)
    calculate_utility_values(model1, utility_node_name_model1, noisy_parameters1)

    return advisor_model, noisy_parameters1



#!!!!!********calling functions to show how to use the operations***********!!!!!!!!!

            
student_model=decision_alternative_transfer(student_model, advisor_model,"course")
student_model=chance_node_transfer(student_model, advisor_model,"content")
advisor_model, noisy_parameters1 = preference_transfer(advisor_model, student_model, parametersA, parametersS, 'friends')

#student_model=chance_node_transfer(student_model, advisor_model,"workload")
#student_model=chance_node_transfer(student_model, advisor_model,"career_development")
#student_model=chance_node_transfer(student_model, advisor_model,"prerequisite")

advisor_model, noisy_parameters2 = preference_transfer(advisor_model, student_model, noisy_parameters1, parametersS, 'Grade')
advisor_model = fill_model1_cpts_with_noisy_data(advisor_model, student_model, 'difficulty')

advisor_model=delete_unmatched_arcs(advisor_model, student_model)

utilities_model_student2, max_expected_utility_student2,_ = show_decision_utilities3(student_model)                
utilities_model_advisor2, max_expected_utility_advisor2,_ = show_decision_utilities3(advisor_model)   
result_df2 = compare_final_decisions(utilities_model_student2, utilities_model_advisor2, threshold=5)
print(result_df2)

gnb.sideBySide(advisor_model,student_model)




Options for decision node 'course' before updating: ('Course1', 'Course2')
Updated options for decision node 'course': ('Course1', 'Course2', 'Course3')
Node content added to the model1 model with its parents.
Child node in model2 is not a utility node.
Node friends added to the model1 model with its parents.
Arc added between friends and Advisor Utility in model1.
Node Grade added to the model1 model with its parents.
Arc added between Grade and Advisor Utility in model1.
Student Model: {'Course1': 25.03, 'Course2': 28.01, 'Course3': 28.63}
Advisor Model: {'Course1': 55.55, 'Course2': 55.07, 'Course3': 57.84}
+-----------------------+----------------------------------------------------------+
|    The Comparisons    |                          Result                          |
+-----------------------+----------------------------------------------------------+
| Max Utility Decisions | Student: ('Course3', 28.63), Advisor: ('Course3', 57.84) |
|         Result        |                 